# Word Embeddings : le modèle Word2Vec

## Imports

In [17]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [18]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [19]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [20]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [21]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [22]:
len(bigram_phrases.vocab.keys())

15761694

Prenons une clé au hasard :

In [23]:
key_ = list(bigram_phrases.vocab.keys())[2]
print(key_)

raege_quotidien


Le dictionnaire indique le score de cette coocurrence :

In [24]:
bigram_phrases.vocab[key_]

1

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [25]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [26]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [27]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [28]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [29]:
print(corpus[:100])

[['raege', 'quotidien', 'j00', 'exemplaires_distribues', 'gratuitement', '#', 'k', '.', "'", 'dans', 'ilagg', ';', 'lomeration', 'bruxelloise', 'et', '.', 'en_province', '.'], ['i', 'personnes_habitant', 'aux_etages', 'fenile', 'journal_contre', '60', 'c', '"<<', 'par_mois', '.'], ['fspir', 'est', 'expedie', 'en', 'proyinoe', 'contre', 't', '*>>"', 'par_mois', ',', 'payables_par_trimestre', '.'], ['annonces', ',', 'faits_divers', 'et', 'reclames', 'sjont', 'recu', 'frie', 'd', "'", 'isabelle', ';', 'a', 'l', "'", 'agence', 'rossfjl', '.'], ['44', '.'], ['-', 'v', ',', 'et', 'do', 'l', "'", 'enseignement', ',', 'a', 'braxell', 'annonces_remises_avant', 'midi', 'sont', ';', '-', 'a', ";'%", '>', '1', 'v', '>>^.'], [':', 'vv', '*', "*'*", 'hv', '/', 'f', '#-', 'v', '1', 'v', ';', "'", 'y', '>', 'fif', "'", 'ji', 'l', "'", 'h', '-', 'et', '.'], ['jeudi', '22', 'decembre', 'i887', 'observatoire', 'roy', "&'", 'cs', 'io', 'decembre', 's', 'heurts', 'uat', '$', 'r', '-:', 'r', 'quotidien_grat

## Entrainement d'un modèle Word2Vec sur ce corpus

In [30]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: user 4min 41s, sys: 12.2 s, total: 4min 53s
Wall time: 1min 57s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [31]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [32]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [33]:
model.wv["ministre"]

array([ 2.2340138 ,  1.6625527 ,  5.7779565 ,  7.852651  ,  0.9210831 ,
        4.3799977 ,  1.2973148 , -2.9736278 , -1.4277197 , -2.2000828 ,
        1.0830339 ,  2.080807  , -0.05076942,  4.8126664 , -4.4216237 ,
       -3.6722047 ,  2.383553  ,  3.5251036 , -1.4651842 ,  0.10838599,
       -5.468068  ,  8.917711  , -2.4472418 ,  4.6378903 ,  0.7680686 ,
        1.2658454 , -2.2989109 , -1.1682074 , -1.4226762 , -4.693583  ,
        5.4783316 ,  1.0490428 ], dtype=float32)

### Calculer la similarité entre deux termes

In [34]:
model.wv.similarity("ministre", "roi")

0.6011553

### Chercher les mots les plus proches d'un terme donné

In [35]:
model.wv.most_similar("ministre", topn=10)

[('president', 0.926480770111084),
 ('president_du_conseil', 0.900627076625824),
 ('commissaire_general', 0.8839254975318909),
 ('ministre_des_finances', 0.8829478621482849),
 ('secretaire_general', 0.8795281648635864),
 ('rapporteur', 0.8793745040893555),
 ('ministre_des_colonies', 0.8735176920890808),
 ('premier_ministre', 0.8710988163948059),
 ('president_du_senat', 0.8699434399604797),
 ('conseiller', 0.858482837677002)]

### Faire des recherches complexes à travers l'espace vectoriel

In [36]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))

[('berlin', 0.8055557012557983), ('rome', 0.7775698304176331), ('datee_du', 0.7602110505104065), ('alger', 0.7545293569564819), ('teheran', 0.7528859376907349), ('tokio', 0.7487483024597168), ('berlin_berlin', 0.7470034956932068), ('jeudi_prochain', 0.741966724395752), ('vienne', 0.7383818626403809), ('new_york', 0.7339268922805786)]
